In [14]:
import logging
import requests
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes
import asyncio

# Logging Configuration
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='stockbot.log',  # Log to a file
    filemode='a'             # Append to the log file
)
logger = logging.getLogger(__name__)

# API Keys
ALPHA_VANTAGE_API_KEY = 'EECJXRG8UJMAGE6M'
TELEGRAM_BOT_TOKEN = '6709242892:AAE96KVSuKfw3ZiCLcCfZgmfSeEmoCbEQyQ'


# --- Command Handlers ---

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print("Start command received") 
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Welcome to the Stock Bot! Use /price <symbol> to get the current stock price.")

async def get_price(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if len(context.args) == 0:
        await context.bot.send_message(chat_id=update.effective_chat.id, text="Please provide a stock symbol (e.g., /price AAPL)")
        return

    symbol = context.args[0].upper()
    url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}'

    try:
        response = requests.get(url)
        data = response.json()

        if 'Error Message' in data:
            await context.bot.send_message(chat_id=update.effective_chat.id, text='Invalid stock symbol or API error.')
        elif data["Global Quote"]["05. price"] == "0.0000":
            await context.bot.send_message(chat_id=update.effective_chat.id, text='Symbol not found or market closed.')
        else:
            price = data["Global Quote"]["05. price"]
            await context.bot.send_message(chat_id=update.effective_chat.id, text=f'Current price of {symbol}: ${price}')

    except requests.exceptions.RequestException as e:
        logger.error(f"Request error: {e}")
        await context.bot.send_message(chat_id=update.effective_chat.id, text='Error fetching data. Please try again later.')
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        await context.bot.send_message(chat_id=update.effective_chat.id, text='An unexpected error occurred. Please try again later.')


# --- Main Function ---

async def main() -> None:
    """Start the bot."""
    application = ApplicationBuilder().token(TELEGRAM_BOT_TOKEN).build()

    application.add_handler(CommandHandler('start', start))
    application.add_handler(CommandHandler('price', get_price))

    await application.initialize()
    await application.start()
    await application.updater.start_polling()

if __name__ == '__main__':
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop